In [30]:
from functools import partial
import logging

import pandas as pd, dask.dataframe as dd
import numpy as np

from empyrical import sharpe_ratio

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (28, 6)
plt.style.use("ggplot")

from utils import stationarity_test
from residualisation import rolling_df_to_series, fa_residualise, pca_residualise

logger = logging.getLogger(__name__)

In [31]:
USECOLS = usecols=("time", "open", "close", "high", "low", "volume", "pair_name", "date")

In [32]:
data = dd.read_csv("data/*", usecols=USECOLS)

daily_volumes = data.groupby(["date", "pair_name"]).sum().compute()
daily_volumes = daily_volumes.reset_index().pivot(values="volume", index="date", columns="pair_name")

daily_prices = data.groupby(["date", "pair_name"]).last().compute()
daily_prices = daily_prices.reset_index().pivot(values="close", index="date", columns="pair_name")

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import yfinance as yf

ticker= "tsla"
tsla_data = yf.download(ticker, start="2018-01-01", end="2022-10-14")

[*********************100%***********************]  1 of 1 completed


In [ ]:
tsla_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,20.799999,21.474001,20.733334,21.368668,21.368668,65283000
2018-01-03,21.400000,21.683332,21.036667,21.150000,21.150000,67822500
2018-01-04,20.858000,21.236668,20.378668,20.974667,20.974667,149194500
2018-01-05,21.108000,21.149332,20.799999,21.105333,21.105333,68868000
2018-01-08,21.066668,22.468000,21.033333,22.427334,22.427334,147891000
...,...,...,...,...,...,...
2022-10-07,233.940002,234.570007,222.020004,223.070007,223.070007,83916800
2022-10-10,223.929993,226.990005,218.360001,222.960007,222.960007,67925000
2022-10-11,220.949997,225.750000,215.000000,216.500000,216.500000,77013200


In [ ]:
log_returns = np.log(1 + daily_prices.pct_change())

GARCH is used to model (log) returns over time since, unlike traditional (S)AR(I)MA modelling, it accounts for heteroskedasticity which is often present due to the fluctuating volatility of financial markets. 

ARCH 

We should check that the variance of our returns are indeed autocorrelated.

In [ ]:
log_returns.rolling(WINDOW, WINDOW // 2).std().plot()

In [40]:
from arch import arch_model
from arch.__future__ import reindexing
from residualisation import rolling_df_to_series

def garch_forecast(y, **garch_kwargs):
    garch = arch_model(y, rescale=False, **garch_kwargs)
    fitted_garch = garch.fit(disp="off")
    return fitted_garch.forecast(horizon=1).variance.values[-1]


def garch_multi_forecast(arr, **garch_kwargs):
    return np.apply_along_axis(garch_forecast, axis=0, arr=arr[:-1])


out = rolling_df_to_series(df, lambda d: garch_multi_forecast(d, rescale = False), window=100)

100%|██████████| 1104/1104 [00:47<00:00, 23.26it/s]


In [ ]:
from arch import arch_model
import numpy as np

fn = lambda s: arch_model(s, vol="garch", p=1, o=0, q=1).fit().forecast(horizon=1).variance.values[-1, :]
df = pd.concat([tsla_data.Close.pct_change().dropna(), 2*tsla_data.Close.pct_change().dropna()], axis=1)
display(df)
display(np.apply_along_axis(fn, axis=0, arr=df))

,Close,Close
Date,,
2018-01-03,-0.010233,-0.020466
2018-01-04,-0.008290,-0.016580
2018-01-05,0.006230,0.012459
2018-01-08,0.062638,0.125276
2018-01-09,-0.008085,-0.016171
...,...,...
2022-10-07,-0.063243,-0.126486
2022-10-10,-0.000493,-0.000986
2022-10-11,-0.028974,-0.057948


Iteration:      1,   Func. Count:      6,   Neg. LLF: 157578721.1857228
Iteration:      2,   Func. Count:     17,   Neg. LLF: 723202262.9682336
Iteration:      3,   Func. Count:     25,   Neg. LLF: -2200.003905010571
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2200.0039049190036
            Iterations: 7
            Function evaluations: 25
            Gradient evaluations: 3
Iteration:      1,   Func. Count:      6,   Neg. LLF: 8638271589.918297
Iteration:      2,   Func. Count:     17,   Neg. LLF: 21517.090307136103
Iteration:      3,   Func. Count:     26,   Neg. LLF: -1225.8617485773502
Iteration:      4,   Func. Count:     32,   Neg. LLF: -1109.4978626739994
Iteration:      5,   Func. Count:     38,   Neg. LLF: -1351.6364232920555
Iteration:      6,   Func. Count:     44,   Neg. LLF: -1319.581193336596
Iteration:      7,   Func. Count:     50,   Neg. LLF: -1350.1417799916449
Iteration:      8,   Func. Count:     56,   Neg. LLF: -1328.

/home/Alex/github_portfolio/crypto_tsa/env/lib/python3.9/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001686. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/Alex/github_portfolio/crypto_tsa/env/lib/python3.9/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/home/Alex/github_portfolio/crypto_tsa/env/lib/python3.9/site-packages/arch/univariate/base.py:

array([[0.00138627, 0.00568751]])

In [ ]:
def residualise(arr, n_components, decomp_model):
    fa = decomp_model(n_components)
    factors = fa.fit_transform(arr[:-1])
    linear_model = LinearRegression()
    linear_model.fit(factors, arr[:-1])
    return arr[-1] - linear_model.predict(fa.transform(arr[-1].reshape(1, -1)))

In [ ]:


def garch_vol_prediction(arr, **garch_kwargs):
    garch = arch_model(arr[:-1], **garch_kwargs)
    fitted_garch = garch.fit(disp="off")
    pred = fitted_garch.forecast(horizon=1)


rolling_df_to_series(log_returns, predict_vol)

In [ ]:
garch_fitted

                     Constant Mean - GARCH Model Results                      
Dep. Variable:                  Close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                2200.00
Distribution:                  Normal   AIC:                          -4392.01
Method:            Maximum Likelihood   BIC:                          -4371.63
                                        No. Observations:                 1204
Date:                Sat, Jan 14 2023   Df Residuals:                     1203
Time:                        21:33:54   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu         2.4845e-03  1.087e-03      2.287  2.222e-02 [3.

In [ ]:
garch_forecast = garch_fitted.forecast(horizon=1)
predicted_et = garch_forecast.mean["h.1"].iloc[-1]